In [2]:
import pickle
import pandas as pd

def get_class_name(filename):
    if 'cat' in filename:
        class_name = 0
    else:
        class_name = 1 
    return class_name

def load_data(dataset='Train'):    
    return pd.read_pickle('../data_processed/' + dataset + '_MELandDeltas.pkl')

def get_dimensions(shape='mel_only', frames=None):
    if shape=='mel_only':            
        mel_height = 128
        mel_depth = 1
    elif shape=='mel_delta':
        mel_height = 256
        mel_depth = 1        
    elif shape=='mel_delta_stacked':
        mel_height = 128
        mel_depth = 2        
    mel_width = int(frames.shape[0]/mel_height/mel_depth)
    return mel_height, mel_width, mel_depth    

In [207]:
def extract_mel_spectrograms(dataset='Train', features=['Mel'], shape='mel_only', window_size=28):
    
    df = load_data(dataset)
        
    #Where it will be stored
    files = []
    labels = []
    data = []
    
    #List of file names in the dataset
    file_names = list(df.File_id.unique())
    
    for file in file_names:
        
        class_name = get_class_name(file)        
        #Filter for the file and extract needed features        
        frames = np.array(df[df['File_id'] == file][features])
        frames = frames.ravel()
        frames = np.concatenate(frames)
        
        #obtain some dimentions about the set to load
        mel_height, mel_width, mel_depth = get_dimensions(shape=shape, frames=frames)

        #Combine all the frames into a mel_spectrogram
        try:
            mel = np.reshape(frames, (mel_height, mel_width, mel_depth))
        except ValueError:                      
            print(file)
            mel = np.reshape(frames, (mel_height, mel_width, mel_depth))
            
        
        #each mel needs to be chopped into segments of window_size width
        batch_size = int(mel.shape[1] / window_size)
        for i in list(range(batch_size)):
            labels.append(class_name)
            files.append(file)            
            data.append(mel[:,i*window_size:(i+1)*window_size])
            
    return np.array(data, dtype=np.float32), np.array(labels), np.array(files)

In [214]:
x_train, y_train, f_train = extract_mel_spectrograms(dataset='Train', features=['Mel', 'Mel_deltas'], 
                                                    shape='mel_delta_stacked', window_size=28)

In [3]:
from util_functions import load_features_with_deltas_stacking, random_mini_batches
train_data, test_data, train_labels, test_labels = load_features_with_deltas_stacking()

In [5]:
test_data.shape

(0,)

In [5]:
df = load_data(dataset='Train')
df.head()

,Mel,File_id,Label,Mel_deltas
0,"[0.0128705910035, 0.0110958605329, 0.050497601...",cat_1,0,"[-0.00106604305625, -0.00243586016761, -0.0035..."
1,"[0.00807935112262, 0.0152217909003, 0.04300330...",cat_1,0,"[-0.00106604305625, -0.00243586016761, -0.0035..."
2,"[0.00219792173099, 0.048424076075, 0.051875263...",cat_1,0,"[-0.00106604305625, -0.00243586016761, -0.0035..."
3,"[0.001428269911, 0.051228681218, 0.06248152103...",cat_1,0,"[-0.00106604305625, -0.00243586016761, -0.0035..."
4,"[0.00153163470852, 0.025880446008, 0.039362137...",cat_1,0,"[-0.00106604305625, -0.00243586016761, -0.0035..."
